<a href="https://colab.research.google.com/github/giacomoarienti/SAE-based-representation-engineering/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/giacomoarienti/SAE-based-representation-engineering.git SpaRE

In [ ]:
%cd /content/SpaRE

In [ ]:
!sh ./scripts/install.sh

In [ ]:
!sed -i 's/^\(@torch.library.register_fake("torchvision::nms")\)/# \1/' /usr/local/lib/python3.11/dist-packages/torchvision/_meta_registrations.py

In [ ]:
from google.colab import userdata
import os

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [ ]:
!sh ./scripts/run_prepare_eval.sh

In [ ]:
import json, torch

path = './cache_data/prepare_eval/nqswap-gemma-2-9b-1shot-examples-closebook/results.json'
output_path = './cache_data/nqswap-gemma-2-2b-memorised_set'

with open(path, 'r') as f:
	results = json.load(f)

# load results["all_close_book_scores"] to pickle
tensor = torch.tensor(results["all_close_book_scores"])
# get the indexes of 1 elements
indexes = torch.nonzero(tensor).squeeze()
# tensor to set
indexes = set(indexes.tolist())

# save to output path
torch.save(indexes, output_path)

In [ ]:
!sh ./scripts/run_group_prompts.sh

In [ ]:
!sh ./scripts/run_save_grouped_activations.sh

In [ ]:
!sh ./scripts/run_mutual_information_and_expectations.sh

In [ ]:
!python ./scripts/run_spare.py \
  --model_path="google/gemma-2-2b" \
  --data_name="nqswap" \
  --layer_ids 9 10 11 12 \
  --edit_degree=2.0 \
  --select_topk_proportion=0.07 \
  --seed=42 \
  --hiddens_name="grouped_activations" \
  --mutual_information_save_name="mutual_information" \
  --run_use_parameter \
  --run_use_context